In [5]:
!pip install openl3 soundfile tqdm os

  Using cached openl3-0.4.2-py2.py3-none-any.whl


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [6]:
!pip install torchaudio transformers tqdm transformers Wav2Vec2Model Wav2Vec2Processor

ERROR: Could not find a version that satisfies the requirement Wav2Vec2Model (from versions: none)
ERROR: No matching distribution found for Wav2Vec2Model


In [11]:
!pip install numpy

In [12]:
import os
import matplotlib.pyplot as plt
import openl3
import numpy as np
import soundfile as sf
import pandas as pd
import numpy as np
from tqdm import tqdm

ModuleNotFoundError: No module named 'numpy.rec'

In [8]:
!pip install openl3

  Using cached openl3-0.4.2-py2.py3-none-any.whl
  Using cached tensorflow-2.19.0-cp39-cp39-win_amd64.whl.metadata (4.1 kB)
  Using cached kapre-0.3.7-py3-none-any.whl
  Using cached resampy-0.2.2-py3-none-any.whl
  Using cached h5py-3.14.0-cp39-cp39-win_amd64.whl.metadata (2.7 kB)
  Using cached moviepy-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached scikit_image-0.24.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached proglog-0.1.12-py3-none-any.whl.metadata (794 bytes)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached tifffile-2024.8.30-py3-none-any.whl.metadata (31 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gas

In [ ]:
import os
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import pandas as pd
from tqdm import tqdm

# === Paths ===
audio_dir = r"C:\Users\hp\Desktop\solicited_data"
output_csv = r"C:\Users\hp\Desktop\solicited_data\solicited_wav2vec2_embeddings.csv"

# === Load pretrained Wav2Vec2 model and processor ===
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
model.eval()

# === Store results ===
embedding_data = []

# === Process each file ===
for filename in tqdm(os.listdir(audio_dir)):
    if filename.endswith(".wav"):
        filepath = os.path.join(audio_dir, filename)
        try:
            waveform, sr = torchaudio.load(filepath)

            # Resample if needed
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                waveform = resampler(waveform)
                sr = 16000

            # Preprocess
            inputs = processor(waveform.squeeze(), sampling_rate=sr, return_tensors="pt", padding=True)

            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden_state = outputs.last_hidden_state  # [batch, time, feature]
                embedding = last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean across time

            embedding_data.append([filename] + embedding.tolist())

        except Exception as e:
            print(f" Error processing {filename}: {e}")


In [14]:
!pip install transformers

In [ ]:
# === Save as CSV ===
columns = ["filename"] + [f"feat_{i}" for i in range(embedding.shape[0])]
df = pd.DataFrame(embedding_data, columns=columns)
df.to_csv(output_csv, index=False)

print(f"\n solicited_wav2vec2_embeddings saved to: {output_csv}")